In [1]:
import visualize.utils
from models import ConvAttnNet
from models import convnext
from models import resnet
import models
# import networks
from visualize.grad_cam import GradCAM
import visualize.utils as utils
from visualize.plot_utils import plot_confmat, plot_gradcam
from torch.autograd import Variable

import torch
import cv2
import pandas as pd
import numpy as np
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from Datasets.datasets import MyDataset
import matplotlib.pyplot as plt

In [2]:
model = convnext.convnext_small(num=1)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

##### Load snapshot

In [4]:
dataset_name = 'ENF'
models.load_snapshot(model, dataset_name, '20220406_205617', 3)
model.cuda()
model.eval()

ConvNeXt(
  (downsample_layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm()
    )
    (1): Sequential(
      (0): LayerNorm()
      (1): Conv2d(96, 192, kernel_size=(2, 2), stride=(2, 2))
    )
    (2): Sequential(
      (0): LayerNorm()
      (1): Conv2d(192, 384, kernel_size=(2, 2), stride=(2, 2))
    )
    (3): Sequential(
      (0): LayerNorm()
      (1): Conv2d(384, 768, kernel_size=(2, 2), stride=(2, 2))
    )
  )
  (stages): ModuleList(
    (0): Sequential(
      (0): Block(
        (dwconv): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
        (norm): LayerNorm()
        (pwconv1): Linear(in_features=96, out_features=384, bias=True)
        (act): GELU()
        (pwconv2): Linear(in_features=384, out_features=96, bias=True)
        (drop_path): Identity()
      )
      (1): Block(
        (dwconv): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)


##### Load weights

In [7]:
# save_path = './weights/image_22k/convnext_base_22k_224.pth'
save_path = './weights/image_1k/convnext_base_1k_224_ema.pth'
checkpoint = torch.load(save_path, map_location=device)
model.load_state_dict(checkpoint["model"])
model.cuda()
model.eval()

ConvNeXt(
  (downsample_layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm()
    )
    (1): Sequential(
      (0): LayerNorm()
      (1): Conv2d(128, 256, kernel_size=(2, 2), stride=(2, 2))
    )
    (2): Sequential(
      (0): LayerNorm()
      (1): Conv2d(256, 512, kernel_size=(2, 2), stride=(2, 2))
    )
    (3): Sequential(
      (0): LayerNorm()
      (1): Conv2d(512, 1024, kernel_size=(2, 2), stride=(2, 2))
    )
  )
  (stages): ModuleList(
    (0): Sequential(
      (0): Block(
        (dwconv): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
        (norm): LayerNorm()
        (pwconv1): Linear(in_features=128, out_features=512, bias=True)
        (act): GELU()
        (pwconv2): Linear(in_features=512, out_features=128, bias=True)
        (drop_path): Identity()
      )
      (1): Block(
        (dwconv): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), g

In [5]:
dirs_train = 'E:/datasets/project1/voxceleb_clip/spec/positive/-15dB/'
dirs_test = 'E:/datasets/project1/voxceleb_clip/spec/positive/-20dB/'
train_set = datasets.ImageFolder(root=dirs_train, transform=utils.transform['eval'])
test_set = datasets.ImageFolder(root=dirs_test, transform=utils.transform['eval'])
train_dl = DataLoader(train_set, batch_size=1)
test_dl = DataLoader(test_set, batch_size=1)

In [6]:
train_preds = utils.get_all_preds(model, train_dl)
test_preds = utils.get_all_preds(model, test_dl)

train_preds.shape, test_preds.shape

(torch.Size([7976, 1]), torch.Size([4228, 1]))

In [42]:
correct = torch.zeros(1).squeeze().cuda()
total = torch.zeros(1).squeeze().cuda()
for i, (images, labels) in enumerate(train_dl):
    
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())

            output = model(images).cuda()
            output = torch.sigmoid(output).sum()
            # print(output)
            if output>=0.5:
                pred = 1
            pred = 0
            if pred == labels:
                correct += 1
            
            total += 1
acc_str = 'Accuracy: %f'%((correct/total).cpu().detach().data.numpy())

In [7]:
train_correct = utils.get_num_correct(train_preds, torch.as_tensor(train_set.targets, device=device))
test_correct = utils.get_num_correct(test_preds, torch.as_tensor(test_set.targets, device=device))

print(f'Train Correct: {train_correct:5}\tTrain Accuracy: {(100*train_correct/len(train_set)):5.2f}%')
print(f'Test Correct: {test_correct:6}\tTest Accuracy: {(100*test_correct/len(test_set)):6.2f}%')

Train Correct:  4084	Train Accuracy: 51.20%
Test Correct:   2256	Test Accuracy:  53.36%


In [16]:
train_correct = utils.get_num_correct(train_preds, torch.as_tensor(train_set.targets, device=device))
test_correct = utils.get_num_correct(test_preds, torch.as_tensor(test_set.targets, device=device))

print(f'Train Correct: {train_correct:5}\tTrain Accuracy: {(100*train_correct/len(train_set)):5.2f}%')
print(f'Test Correct: {test_correct:6}\tTest Accuracy: {(100*test_correct/len(test_set)):6.2f}%')

Train Correct:  6297	Train Accuracy: 85.44%
Test Correct:   6410	Test Accuracy:  80.37%


In [19]:
train_correct = utils.get_num_correct(train_preds, torch.as_tensor(train_set.targets, device=device))
test_correct = utils.get_num_correct(test_preds, torch.as_tensor(test_set.targets, device=device))

print(f'Train Correct: {train_correct:5}\tTrain Accuracy: {(100*train_correct/len(train_set)):5.2f}%')
print(f'Test Correct: {test_correct:6}\tTest Accuracy: {(100*test_correct/len(test_set)):6.2f}%')

Train Correct:  3562	Train Accuracy: 85.38%
Test Correct:   3485	Test Accuracy:  82.43%


In [10]:
train_confmat = utils.get_confmat(train_set.targets, train_preds)
test_confmat = utils.get_confmat(test_set.targets, test_preds)
plot_confmat(train_confmat, test_confmat, train_set.classes, f'{type(convnext).__name__.lower()}')

IndexError: index 2 is out of bounds for dimension 0 with size 2

In [ ]:
results = utils.get_results(test_confmat, test_set.classes)
results